In [1]:
import pandas as pd
import numpy as np
import json
from haversine import haversine
import os
import datetime
import zipcodes
# Use 2018 Q4 quadrant defination and only include stores labeled with quadrant

In [2]:
zips_by_store=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",dtype=str)
zips_by_store=zips_by_store[['location_id','zip','revenue_flag']].drop_duplicates()
zips_by_store=zips_by_store[zips_by_store['revenue_flag'].isin(['P','S'])]
zips_by_store=zips_by_store.rename(columns={"zip":"zip_cd"})
zips_by_store['zip_cd']=zips_by_store['zip_cd'].apply(lambda x: x.zfill(5)) # Though all 5 digits
zips_by_store=zips_by_store[zips_by_store['location_id']!="6990"]

In [3]:
Q4_Quadrant=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/zips_by_quadrant_20190416/Excel_BL_2018_Q4_post_YoY_small_JL_2019-03-04.xlsx",
                          dtype=str,sheetname="Q4_Store_Quadrant_Defination",usecols=['location_id','Quadrant'])

print(Q4_Quadrant.shape)
print(len(Q4_Quadrant['location_id'].unique()))
Q4_Quadrant_and_zips=pd.merge(Q4_Quadrant,zips_by_store,on="location_id",how="left")
print(len(Q4_Quadrant_and_zips['location_id'].unique()))
print(len(Q4_Quadrant_and_zips[~pd.isnull(Q4_Quadrant_and_zips['revenue_flag'])]['location_id'].unique()))
print(len(Q4_Quadrant_and_zips[pd.isnull(Q4_Quadrant_and_zips['revenue_flag'])]['location_id'].unique()))



(1361, 2)
1361
1361
1337
24


In [4]:
Q4_Quadrant_and_zips.head(2)

,location_id,Quadrant,zip_cd,revenue_flag
0,1,Quadrant I,43232,P
1,1,Quadrant I,43209,P


In [5]:
new_stores=Q4_Quadrant_and_zips[pd.isnull(Q4_Quadrant_and_zips['revenue_flag'])]
Q4_Quadrant_and_zips=Q4_Quadrant_and_zips[~pd.isnull(Q4_Quadrant_and_zips['revenue_flag'])]
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190401-134939-117.txt",sep="|",dtype=str)

store_list=store_list[['location_id','latitude_meas','longitude_meas']]
new_stores=pd.merge(new_stores,store_list,on="location_id",how="left")
new_stores['latitude_meas']=new_stores['latitude_meas'].astype(float)
new_stores['longitude_meas']=new_stores['longitude_meas'].astype(float)
new_stores=new_stores.reset_index()
del new_stores['index']



In [6]:
stores_open_no_quadrant=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2019-04-06.csv",usecols=['week_end_date','location_id','sales'],dtype=str)
stores_open_no_quadrant=stores_open_no_quadrant[stores_open_no_quadrant['week_end_date'].isin(['2019-04-06','2019-03-30','2019-03-23','2019-03-16'])]
stores_open_no_quadrant['sales']=stores_open_no_quadrant['sales'].astype(float)
stores_open_no_quadrant=stores_open_no_quadrant[stores_open_no_quadrant['sales']>0]
stores_open_no_quadrant=stores_open_no_quadrant.groupby(['location_id'])['sales'].count().to_frame().reset_index().rename(columns={"sales":"count"})
stores_open_no_quadrant=stores_open_no_quadrant[stores_open_no_quadrant['count']>=3]
stores_open_no_quadrant=stores_open_no_quadrant.reset_index()
del stores_open_no_quadrant['index']
print(stores_open_no_quadrant.shape)

stores_open_no_quadrant=stores_open_no_quadrant[['location_id']]
stores_open_no_quadrant['Quadrant']='Quadrant III'
stores_open_no_quadrant=pd.merge(stores_open_no_quadrant,store_list,on="location_id",how="left")
stores_open_no_quadrant=stores_open_no_quadrant[~stores_open_no_quadrant['location_id'].isin(Q4_Quadrant_and_zips['location_id'].unique().tolist())]
stores_open_no_quadrant=stores_open_no_quadrant[~stores_open_no_quadrant['location_id'].isin(new_stores['location_id'].unique().tolist())]
print(stores_open_no_quadrant.shape)

stores_open_no_quadrant['latitude_meas']=stores_open_no_quadrant['latitude_meas'].astype(float)
stores_open_no_quadrant['longitude_meas']=stores_open_no_quadrant['longitude_meas'].astype(float)
new_stores=new_stores.append(stores_open_no_quadrant)

print(new_stores.shape)


new_stores=new_stores.reset_index()
del new_stores['index']

(1398, 2)
(41, 4)
(65, 6)


In [7]:
new_stores['Quadrant'].unique()

array(['Quadrant III', 'Quadrant II', 'Quadrant IV', 'Quadrant I'], dtype=object)

In [8]:
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/original_center_of_rentrak_zip.json","r"))
new_store_10_miles_zips=pd.DataFrame()
i_counter=0
for i in range(len(new_stores)):
    store_id=new_stores['location_id'][i]
    store_center=[new_stores['latitude_meas'][i],new_stores['longitude_meas'][i]]
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
        if dist<=10:
            i_counter+=1
            df=pd.DataFrame({"location_id":store_id,"zip_cd":zip_cd,"revenue_flag":"10_miles"},index=[i_counter])
            new_store_10_miles_zips=new_store_10_miles_zips.append(df)
new_store_10_miles_zips=pd.merge(new_store_10_miles_zips,Q4_Quadrant,on="location_id",how="left")
new_store_10_miles_zips['Quadrant']=new_store_10_miles_zips['Quadrant'].fillna("Quadrant III")

In [9]:
Q4_Quadrant_and_zips.head(2)

,location_id,Quadrant,zip_cd,revenue_flag
0,1,Quadrant I,43232,P
1,1,Quadrant I,43209,P


In [10]:
all_zips_quadrant_flag=Q4_Quadrant_and_zips.append(new_store_10_miles_zips)
all_zips_quadrant_flag=pd.merge(all_zips_quadrant_flag,store_list,on="location_id",how="left")
all_zips_quadrant_flag['latitude_meas']=all_zips_quadrant_flag['latitude_meas'].astype(float)
all_zips_quadrant_flag['longitude_meas']=all_zips_quadrant_flag['longitude_meas'].astype(float)
i_counter=0
all_zips_quadrant_flag=all_zips_quadrant_flag.reset_index()
del all_zips_quadrant_flag['index']
all_zips_quadrant_flag['dist_miles']=np.nan
for i in range(len(all_zips_quadrant_flag)):
    zip_cd=all_zips_quadrant_flag['zip_cd'][i]
    store_center=[all_zips_quadrant_flag['latitude_meas'][i],all_zips_quadrant_flag['longitude_meas'][i]]
    if zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
    else:
        try:
            zip_center=[float(zipcodes.matching(zip_cd)['lag']),float(zipcodes.matching(zip_cd)['long'])]
            dist=haversine(store_center,zip_center,miles=True)
        except:
            dist=np.nan
    all_zips_quadrant_flag['dist_miles'][i]=dist
    i_counter+=1
    if i%1000==11:
        print(datetime.datetime.now(),i)
all_zips_quadrant_flag=all_zips_quadrant_flag[(all_zips_quadrant_flag['revenue_flag']!="S") | (all_zips_quadrant_flag['dist_miles']<=50)]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2019-04-16 14:28:24.590134 11
2019-04-16 14:28:46.319286 1011
2019-04-16 14:29:08.359878 2011
2019-04-16 14:29:30.189776 3011
2019-04-16 14:29:52.352330 4011
2019-04-16 14:30:14.329151 5011
2019-04-16 14:30:36.549980 6011
2019-04-16 14:30:58.669732 7011
2019-04-16 14:31:20.687592 8011
2019-04-16 14:31:42.686876 9011
2019-04-16 14:32:04.770092 10011
2019-04-16 14:32:26.938116 11011
2019-04-16 14:32:49.098609 12011
2019-04-16 14:33:11.229852 13011
2019-04-16 14:33:33.318118 14011
2019-04-16 14:33:55.410818 15011
2019-04-16 14:34:17.285730 16011
2019-04-16 14:34:39.378293 17011
2019-04-16 14:35:01.269482 18011
2019-04-16 14:35:23.101982 19011
2019-04-16 14:35:44.970876 20011
2019-04-16 14:36:06.945420 21011
2019-04-16 14:36:28.906736 22011
2019-04-16 14:36:50.805695 23011
2019-04-16 14:37:12.243911 24011
2019-04-16 14:37:33.574126 25011


In [11]:
# Priority: the higher, the better
all_zips_quadrant_flag['quadrant_priority']=all_zips_quadrant_flag['Quadrant'].replace("Quadrant III",4).replace("Quadrant II",3).replace("Quadrant IV",2).replace("Quadrant I",1)
all_zips_quadrant_flag['flag_priority']=all_zips_quadrant_flag['revenue_flag'].replace("P",3).replace("S",2).replace("10_miles",1)
all_zips_quadrant_flag['priority_sort']=all_zips_quadrant_flag['quadrant_priority']*10+all_zips_quadrant_flag['flag_priority']

all_zips_quadrant_flag=all_zips_quadrant_flag.sort_values(['zip_cd','priority_sort'],ascending=[True,False])



In [12]:
def count_unique(x):
    y=len(set(x))
    return y

In [13]:
all_zips_quadrant_flag_deduped=all_zips_quadrant_flag.drop_duplicates(['zip_cd'])
summary_after_dedup=all_zips_quadrant_flag_deduped.groupby(['Quadrant','revenue_flag'])['zip_cd'].count().to_frame().reset_index().rename(columns={"zip_cd":"zip_count"})

In [14]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/zips_by_quadrant_20190416/BL_zips_by_quadrant_S_in_50_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
all_zips_quadrant_flag_deduped.to_excel(writer,"deduped_zips",index=False)
summary_after_dedup.to_excel(writer,"deduped_summary",index=False)
all_zips_quadrant_flag.to_excel(writer,"zips_by_store_in_Q3_Quadrants",index=False)
writer.save()
